In [1]:
import os
import numpy as np
import pandas as pd
import torch
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader

In [2]:
train = pd.read_csv('/kaggle/input/hms-harmful-brain-activity-classification/train.csv')
print('Train shape', train.shape )
train

Train shape (106800, 15)


,eeg_id,eeg_sub_id,eeg_label_offset_seconds,spectrogram_id,spectrogram_sub_id,spectrogram_label_offset_seconds,label_id,patient_id,expert_consensus,seizure_vote,lpd_vote,gpd_vote,lrda_vote,grda_vote,other_vote
0,1628180742,0,0.0,353733,0,0.0,127492639,42516,Seizure,3,0,0,0,0,0
1,1628180742,1,6.0,353733,1,6.0,3887563113,42516,Seizure,3,0,0,0,0,0
2,1628180742,2,8.0,353733,2,8.0,1142670488,42516,Seizure,3,0,0,0,0,0
3,1628180742,3,18.0,353733,3,18.0,2718991173,42516,Seizure,3,0,0,0,0,0
4,1628180742,4,24.0,353733,4,24.0,3080632009,42516,Seizure,3,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
106795,351917269,6,12.0,2147388374,6,12.0,4195677307,10351,LRDA,0,0,0,3,0,0
106796,351917269,7,14.0,2147388374,7,14.0,290896675,10351,LRDA,0,0,0,3,0,0
106797,351917269,8,16.0,2147388374,8,16.0,461435451,10351,LRDA,0,0,0,3,0,0
106798,351917269,9,18.0,2147388374,9,18.0,3786213131,10351,LRDA,0,0,0,3,0,0


In [3]:
TARGETS = train.columns[-6:]

train_17k = train.groupby('eeg_id')[['spectrogram_id','spectrogram_label_offset_seconds']].agg(
    {'spectrogram_id':'first','spectrogram_label_offset_seconds':'min'})
train_17k.columns = ['spec_id','min']

tmp = train.groupby('eeg_id')[['spectrogram_id','spectrogram_label_offset_seconds']].agg(
    {'spectrogram_label_offset_seconds':'max'})
train_17k['max'] = tmp

tmp = train.groupby('eeg_id')[['patient_id']].agg('first')
train_17k['patient_id'] = tmp

tmp = train.groupby('eeg_id')[TARGETS].agg('sum')
for t in TARGETS:
    train_17k[t] = tmp[t].values
    
y_data = train_17k[TARGETS].values
y_data = y_data / y_data.sum(axis=1,keepdims=True)
train_17k[TARGETS] = y_data

tmp = train.groupby('eeg_id')[['expert_consensus']].agg('first')
train['target'] = tmp

train_17k = train_17k.reset_index()
print('Train non-overlapp eeg_id shape:', train_17k.shape )
train_17k

Train non-overlapp eeg_id shape: (17089, 11)


,eeg_id,spec_id,min,max,patient_id,seizure_vote,lpd_vote,gpd_vote,lrda_vote,grda_vote,other_vote
0,568657,789577333,0.0,16.0,20654,0.0,0.000000,0.25,0.000000,0.166667,0.583333
1,582999,1552638400,0.0,38.0,20230,0.0,0.857143,0.00,0.071429,0.000000,0.071429
2,642382,14960202,1008.0,1032.0,5955,0.0,0.000000,0.00,0.000000,0.000000,1.000000
3,751790,618728447,908.0,908.0,38549,0.0,0.000000,1.00,0.000000,0.000000,0.000000
4,778705,52296320,0.0,0.0,40955,0.0,0.000000,0.00,0.000000,0.000000,1.000000
...,...,...,...,...,...,...,...,...,...,...,...
17084,4293354003,1188113564,0.0,0.0,16610,0.0,0.000000,0.00,0.000000,0.500000,0.500000
17085,4293843368,1549502620,0.0,0.0,15065,0.0,0.000000,0.00,0.000000,0.500000,0.500000
17086,4294455489,2105480289,0.0,0.0,56,0.0,0.000000,0.00,0.000000,0.000000,1.000000
17087,4294858825,657299228,0.0,12.0,4312,0.0,0.000000,0.00,0.000000,0.066667,0.933333


In [4]:
ids = train.eeg_id.unique()
sample_ids = ids[0:5]
sample_ids

array([1628180742, 2277392603,  722738444,  387987538, 2175806584])

In [5]:
import pywt

def maddest(d, axis=None):
    return np.mean(np.absolute(d - np.mean(d, axis)), axis)

def denoise(x, wavelet='db8', level=1):
    #x = x.iloc[:, 1:-7]
    ret = {key:[] for key in x.columns}
    
    for pos in x.columns:
        coeff = pywt.wavedec(x[pos], wavelet, mode="per")
        sigma = (1/0.6745) * maddest(coeff[-level])

        uthresh = sigma * np.sqrt(2*np.log(len(x)))
        coeff[1:] = (pywt.threshold(i, value=uthresh, mode='hard') for i in coeff[1:])

        ret[pos]=pywt.waverec(coeff, wavelet, mode='per')
    
    return pd.DataFrame(ret)

In [6]:
# DWT
from pywt import wavedec

def wavelet_decompose_channels(data, level, output=False):
  # take every x number of points using numpy's slicing (start:stop:step)
    data = data[0::2]

    data.columns.name='channel'

    # transpose the data
    data_t = data.transpose()

    # get the wavelet coefficients at each level in a list
    coeffs_list = wavedec(data_t.values, wavelet='db4', level=level)
    #print(len(coeffs_list))

    # make a list of the component names (later column rows)
    nums = list(range(1,level+1))
    names=[]
    for num in nums:
        names.append('D' + str(num))
    names.append('A' + str(nums[-1]))

  # reverse the names so it counts down
    names = names[::-1]  
    #print(names)

    i = 0
    wavelets = pd.DataFrame()
    for i in range(1, len(coeffs_list)):
    #for i, array in enumerate(coeffs_list):
        #print(i)
        array = coeffs_list[i]
        # turn into a dataframe and transpose
        level_df = pd.DataFrame(array)
        level_df.index = data.columns
        level_df['level'] = names[i]
        level_df= level_df.set_index('level', append=True)
        level_df=level_df.T
        # add the next levels df to another column
        wavelets = pd.concat([wavelets,level_df], axis=1, sort=True)

    # sort values along the channels
    wavelets = wavelets.sort_values(['channel', 'level'], axis=1)

    wavelets_cleaned = wavelets.dropna()

    return wavelets_cleaned

# usage
#dwt_wavelets = wavelet_decompose_channels(denoised_eeg_data, level=5, output=True)
#dwt_wavelets

In [7]:
from scipy.stats import entropy

def MAV(data):
    # Initialize an empty DataFrame to store the means for each channel
    means = pd.DataFrame(index=data.index)

    # Iterate over each channel and calculate the mean across 'D1' to 'D5'
    for channel in data.columns.get_level_values(0).unique():
    # Calculate the mean for the current channel
        means[channel] = data[channel].abs().mean(axis=1)
    
    means.columns = [f"{col}_DT_MAV" for col in means.columns]
    
    return means


def MAVP(data):
    # Initialize an empty DataFrame to store the means for each channel
    means_abs = pd.DataFrame(index=data.index)

    # Iterate over each channel and calculate the mean across 'D1' to 'D5'
    for channel in data.columns.get_level_values(0).unique():
    # Calculate the mean for the current channel
        means_abs[channel] = (data[channel]**2).mean(axis = 1)
    
    means_abs.columns = [f"{col}_DT_MAVP" for col in means_abs.columns]
    
    return means_abs



def std_val(data):
    # Initialize an empty DataFrame to store the means for each channel
    std_vals = pd.DataFrame(index=data.index)

    # Iterate over each channel and calculate the mean across 'D1' to 'D5'
    for channel in data.columns.get_level_values(0).unique():
    # Calculate the mean for the current channel
        std_vals[channel] = data[channel].std(axis = 1)
    
    std_vals.columns = [f"{col}_DT_STD" for col in std_vals.columns]
    
    return std_vals


def var_val(data):
    # Initialize an empty DataFrame to store the means for each channel
    var = pd.DataFrame(index=data.index)

    # Iterate over each channel and calculate the mean across 'D1' to 'D5'
    for channel in data.columns.get_level_values(0).unique():
    # Calculate the mean for the current channel
        var[channel] = data[channel].var(axis = 1)
    
    var.columns = [f"{col}_DT_VAR" for col in var.columns]
    
    return var


def ratio_channels(epoch_data):
    # Initialize an empty DataFrame to store the ratio values for each pair of channels
    ratio_data = pd.DataFrame(index=epoch_data.index)
    
    # Iterate over each pair of adjacent channels
    for i in range(len(epoch_data.columns) - 1):
        channel1 = epoch_data.columns[i]
        channel2 = epoch_data.columns[i + 1]
        
        # Calculate the ratio between the values of the two channels
        ratio_data[f"{channel1}-{channel2}_Ratio"] = epoch_data[channel1] / epoch_data[channel2]
    
    return ratio_data



def shannon_entropy(data):
    # Initialize an empty DataFrame to store the entropy values for each channel
    entropy_df = pd.DataFrame(index=data.index)

    # Iterate over each channel and calculate the entropy across 'D1' to 'D5'
    for channel in data.columns.get_level_values(0).unique():
        # Calculate the entropy for the current channel
        entropy_values = data[channel].apply(lambda x: entropy(x.abs(), base=2), axis=1)
        entropy_df[channel + '_entropy'] = entropy_values

    return entropy_df

In [8]:
def eeg_feature(data):
    
    wavelet_mean = MAV(data)
    wavelet_meanabs = MAVP(data)
    wavelet_std = std_val(data)
    wavelet_var = var_val(data)
    wavelet_ratio = ratio_channels(data)
    wavelet_entropy = shannon_entropy(data)
    
    wavelet_statistics = pd.concat([data, wavelet_mean, wavelet_meanabs, wavelet_std, wavelet_var, wavelet_ratio, wavelet_entropy], axis=1)
    
    wavelet_statistics.columns = ['_'.join(col).strip() if isinstance(col, tuple) else col for col in wavelet_statistics.columns]

    
    features_to_drop = ['C4_D1', 'F3_D1', 'F4_D1', 'F7_D1', 'F8_D1', 'Fp2_D1', 'Fz_D1', 'O2_D1', 'T3_D1', 'T4_D1', 'T5_D1', 'T6_D1', "('Fp2', 'D1')-('Fp2', 'D2')_Ratio", 'C3_entropy', 'C4_entropy', 'Cz_entropy', 'EKG_entropy', 'F3_entropy', 'F4_entropy', 'F7_entropy', 'F8_entropy', 'Fp1_entropy', 'Fp2_entropy', 'Fz_entropy', 'O1_entropy', 'O2_entropy', 'P3_entropy', 'P4_entropy', 'Pz_entropy', 'T3_entropy', 'T4_entropy', 'T5_entropy', 'T6_entropy']

    # Drop the specified features from the DataFrame
    wavelet_statistics.drop(columns=features_to_drop, inplace=True)
    
    return wavelet_statistics


In [9]:
def feature_eng_eeg(parquet_path, eegid, display=False):
        
    GET_ROW = 0

    row = train.iloc[GET_ROW]

    eeg = pd.read_parquet(parquet_path)
    
    # Denoise
    denoised_eeg_data = denoise(eeg, wavelet='db8') 
    
    # DWT
    dwt_wavelets = wavelet_decompose_channels(denoised_eeg_data, level=5, output=True)
    
    # getting all features
    eegid_data_features = eeg_feature(dwt_wavelets)
    
    print(f"features done {eeg_id}")
    
    return eegid_data_features

In [10]:
%%time
PATH = '/kaggle/input/hms-harmful-brain-activity-classification/train_eegs/'

EEG_IDS = sample_ids
all_eegs = {}
# for i,eeg_id in tqdm(enumerate(EEG_IDS)): # use this when in kaggle notebook
for i,eeg_id in enumerate(EEG_IDS):         # use this when saving output
    if (i%100==0)&(i!=0): 
        print(i,', ',end='')
        
    
    # CREATE FEATURES FROM EEG PARQUET
    all_features_data = feature_eng_eeg(f'{PATH}{eeg_id}.parquet',eeg_id)
    # Remove rows with NaN values in feature data
    clean_features_data = all_features_data.dropna()
    
    if clean_features_data.empty:
        continue 
    
    
    # GET TARGET LABELS FOR SPECIFIC EEF_ID
    train_17k_eegid = train_17k[train_17k['eeg_id'] == eeg_id]
    target_labels = train_17k_eegid.iloc[0][['seizure_vote', 'lpd_vote', 'gpd_vote', 'lrda_vote', 'grda_vote', 'other_vote']]

    # SAVE 
    all_eegs[eeg_id] = (clean_features_data, target_labels)
    

   
# # SAVE EEG SPECTROGRAM DICTIONARY
#np.save('/kaggle/working/full_dataset',all_eegs)

features done 1628180742
features done 2277392603
features done 722738444
features done 387987538
features done 2175806584
CPU times: user 3.48 s, sys: 41.6 ms, total: 3.52 s
Wall time: 3.73 s


In [11]:
len(all_eegs[2175806584][0].columns)

266

In [29]:
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.utils.rnn import pad_sequence
import pandas as pd

In [64]:
class EEGDataset(Dataset):
    def __init__(self, data_dict):
        # Initialize empty lists to hold concatenated data and targets
        concatenated_data = []
        concatenated_targets = []

        # Loop over each item in the dictionary to concatenate data
        for _, (df, target) in data_dict.items():
            # Convert the dataframe to a tensor and append to list
            eeg_data_tensor = torch.tensor(df.values, dtype=torch.float32)
            target_tensor = torch.tensor(target, dtype=torch.float32).repeat(len(df), 1)  # Repeat target for each row in df
            concatenated_data.append(eeg_data_tensor)
            concatenated_targets.append(target_tensor)

        # Concatenate all data tensors and target tensors
        self.data = torch.cat(concatenated_data, dim=0)
        self.targets = torch.cat(concatenated_targets, dim=0)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        # Return a single data point and its target
        return self.data[index], self.targets[index]

In [92]:
dataset = EEGDataset(all_eegs)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

# Example iteration over the DataLoader
for eeg_data, targets in dataloader:
    print(eeg_data.shape)  # Each batch may have different sequence lengths but the same number of features
    print(targets.shape)
    break;

torch.Size([32, 266])
torch.Size([32, 6])


/tmp/ipykernel_33/2157899305.py:11: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  target_tensor = torch.tensor(target, dtype=torch.float32).repeat(len(df), 1)  # Repeat target for each row in df


In [126]:
class CNNModel(nn.Module):
    def __init__(self, num_features, num_classes):
        super(CNNModel, self).__init__()

        self.features = nn.Sequential(
            nn.Conv1d(1, 64, kernel_size=3, padding=1),
            nn.LeakyReLU(),
            nn.Conv1d(64, 64, kernel_size=3, padding=1),
            nn.LeakyReLU(),
            nn.MaxPool1d(2),
            nn.BatchNorm1d(64),

            nn.Conv1d(64, 128, kernel_size=3, padding=1),
            nn.LeakyReLU(),
            nn.Conv1d(128, 128, kernel_size=3, padding=1),
            nn.LeakyReLU(),
            nn.MaxPool1d(2),
            nn.BatchNorm1d(128)
            
            )
        
        # Calculate the number of features after all conv and pooling layers to correctly size the input to the linear layer
        self._to_linear = None
        self._num_features(num_features)

        self.classifier = nn.Sequential(
            nn.Linear(self._to_linear, 512),
            nn.LeakyReLU(),
            nn.Linear(512, num_classes),
            nn.Softmax(dim=1)
        )

    def _num_features(self, size):
        # Forward pass on dummy data to calculate flat features after conv layers
        inputs = torch.rand(1, 1, size)
        outputs = self.features(inputs)
        self._to_linear = int(outputs.numel() / outputs.shape[0])

    def forward(self, x):
        # Assuming x is of shape (batch, features), we unsqueeze to add channel dimension
        x = x.unsqueeze(1)  # Shape becomes (batch, 1, features)
        x = self.features(x)
        x = x.view(x.size(0), -1)  # Flatten the output for the classifier
        x = self.classifier(x)
        return x

# Example usage:
model = CNNModel(num_features=266, num_classes=6)

In [129]:
# Assuming DWT and statistical features are loaded as (batch_size, num_features, sequence_length)
# For example, let's assume 10 statistical features over 100 time intervals
#model = CNNModel(num_features=266, num_classes=6)
optimizer = torch.optim.Adam(model.parameters(), lr=0.000001)
criterion = torch.nn.KLDivLoss(reduction="batchmean")  # Or another appropriate loss function
epochs = 10

In [130]:
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    for i, (inputs, labels) in enumerate(dataloader):
        # Zero the parameter gradients
        optimizer.zero_grad()
        
        #print(labels.shape)

        # Forward pass
        outputs = model(inputs)
        #print(outputs.shape)
    
        # Compute the loss
        loss = criterion(outputs, labels)
        
        # Backward pass and optimize
        loss.backward()
        optimizer.step()

        # Print statistics
        running_loss += loss.item()
        if i % 10 == 9:    # print every 10 mini-batches
            print(f'Epoch {epoch + 1}, Batch {i + 1}: Loss: {running_loss / 10:.4f}')
            running_loss = 0.0

            
print('Finished Training')

Epoch 1, Batch 10: Loss: nan
Epoch 1, Batch 20: Loss: nan
Epoch 1, Batch 30: Loss: nan
Epoch 2, Batch 10: Loss: nan
Epoch 2, Batch 20: Loss: nan
Epoch 2, Batch 30: Loss: nan
Epoch 3, Batch 10: Loss: nan
Epoch 3, Batch 20: Loss: nan


KeyboardInterrupt: 

In [26]:
# Training loop
for epoch in range(epochs):
    for eeg_data, targets in eeg_dataloader:
        print(eeg_data.shape)
        optimizer.zero_grad()
        outputs = model(eeg_data)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
    print(f'Epoch {epoch+1}, Loss: {loss.item()}')

KeyError: Caught KeyError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/site-packages/torch/utils/data/_utils/worker.py", line 308, in _worker_loop
    data = fetcher.fetch(index)
  File "/opt/conda/lib/python3.10/site-packages/torch/utils/data/_utils/fetch.py", line 51, in fetch
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "/opt/conda/lib/python3.10/site-packages/torch/utils/data/_utils/fetch.py", line 51, in <listcomp>
    data = [self.dataset[idx] for idx in possibly_batched_index]
KeyError: 0


In [ ]:
model.compile(optimizer='adam', loss='kullback_leibler_divergence', metrics=['accuracy'])

# 3. Train the model
# Example:
# Train the model using the SWT dataset
history = model.fit(dwt_wavelets_features, replicated_labels_df, epochs=10, batch_size=32, validation_split=0.2)

# 4. Evaluate the model
# Example:
# Evaluate the trained model on the test set
test_loss, test_accuracy = model.evaluate(X_test_normalized, y_test)
print(f'Test Loss: {test_loss}, Test Accuracy: {test_accuracy}')

# Optional: Visualize training history
# Example:
# Plot training and validation accuracy over epochs
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.show()